## Import libraries

In [100]:
import time
import random

import requests
from bs4 import BeautifulSoup

import re
import pandas as pd
from datetime import datetime
import math 
import numpy as np
from sql_functions import *    

import psycopg2    

cycle_number = '03'

# create global list of targeted neighbourhoods AND match with platform location IDs
#neighbourhoods = ['Hammersmith and Fulham', 'Kensington and Chelsea', 'Camden', 'City of Westminster', 'City of London', 'Hackney', 'Lambeth', 'Tower Hamlets', 'Islington']
neighbourhoods = ['Hackney', 'Lambeth', 'Tower Hamlets', 'Islington']
#rightmove_loc_id = ['5E61407', '5E61229', '5E93941', '5E61233', '5E61224', '5E93953', '5E93971', '5E61417', '5E93965']
rightmove_loc_id = ['5E93953', '5E93971', '5E61417', '5E93965']

# rightmove_loc_id = ['5E61224']
# neighbourhoods = ['City of London']

zip_iterator = zip(rightmove_loc_id, neighbourhoods)
locations_dict = dict(zip_iterator)
    # {
    # '5E61407': 'Hammersmith and Fulham',
    # '5E61229': 'Kensington and Chelsea',
    # '5E93941': 'Camden',
    # '5E61233': 'City of Westminster',
    # '5E61224': 'City of London',
    # '5E93953': 'Hackney',
    # '5E93971': 'Lambeth',
    # '5E61417': 'Tower Hamlets',
    # '5E93965': 'Islington'
    # }

################furnishTypes_lst = ['furnished', 'unfurnished', 'partFurnished']

#---------------------------------------------

# get internal IDs
def get_ids_rm(bs):
    ids = bs.find_all(class_ = 'l-searchResult is-list')
    ids_lst = [id.get('id') for id in ids]
    ids_lst = [(str(x)) for x in ids_lst]
    return(ids_lst)
# ids_lst

#------------------------- new new
#-------------------------

# get property type
def get_property_type_rm(bs):
    types_lst = [title.text.strip().split()[-1] for title in bs.findAll('h2', {'class': 'propertyCard-title'})]
    types_lst = list(filter(lambda proptype: proptype != 'Property', types_lst)) # delete any placeholder values looking like "Property"
    return(types_lst)


# # get number of bedrooms
# def get_bedrooms_rm(bs):
#     bedrooms_lst = [title.text.split('bedroom')[0].strip() for title in bs.findAll('h2', {'class': 'propertyCard-title'})]
#     return(bedrooms_lst)

# # get number of bathrooms
# def get_bathrooms_rm(bs):
#     bathrooms = bs.find_all(class_= "propertyCard-features")
#     bathrooms_lst = [bathroom.get_text() for bathroom in bathrooms]
#     bathrooms_lst = list(filter(None, bathrooms_lst)) #delete any empty strings from list
#     # format bathrooms
#     bathrooms_lst = [bathroom.split('bathroom')[0].strip() for bathroom in bathrooms_lst]
#     return(bathrooms_lst)


# function for prices monthly and weekly(=sec_prices)
def get_prices_rm(bs):
    prices = bs.find_all(class_= "propertyCard-priceValue")
    prices_lst = [price.get_text() for price in prices]
    prices_lst = list(filter(None, prices_lst)) #delete any empty strings from list in order to not break list structure structure/ loop
    # format prices
    prices_lst = [price.removesuffix(' pcm').replace('£', '').replace(',', '') for price in prices_lst]
    #prices_lst = [float(price) for price in prices_lst] #comment out to avoid error when no price is found or when price is not a number("POA")
    
    return(prices_lst)  


# def get_sec_prices_rm(bs):
#     sec_prices = bs.find_all(class_= "propertyCard-secondaryPriceValue")
#     sec_prices_lst = [sec_price.get_text() for sec_price in sec_prices]
#     sec_prices_lst = list(filter(None, sec_prices_lst)) #delete any empty strings from list
#     #format sec_prices
#     sec_prices_lst = [sec_price.removesuffix(' pw').replace('£', '').replace(',', '') for sec_price in sec_prices_lst]
#     #sec_prices_lst = [float(sec_price) for sec_price in sec_prices_lst]
#     return(sec_prices_lst)

#---------------------------------------------

def page_results(loc_id):
    # get content from immo website; create souppr
    page = requests.get(
        "https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{}&index={}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&keywords=".format(loc_id, '0'))
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # get number of search results
    num_results = bs.find(class_="searchHeader-resultCount").get_text()
    num_results = num_results.replace(',', '')
    num_results = int(num_results)
    num_pages = math.ceil(num_results / 24) # don't know why, but have to use 24 instead of 25 (results per page)

    # list of page indices we can use for the url to check every single page of search results
    page_indices = [x*24 for x in range(0, num_pages)]

    # ...
    df_search_rm = pd.DataFrame()

    for page_number in page_indices:
        # sleep timer
        time.sleep(random.randint(2,5)/10)

        # ...
        url = "https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{}&index={}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&keywords=".format(loc_id, page_number)
        page = requests.get(url)
        html = page.content
        bs = BeautifulSoup(html, 'html.parser')

        # Create a dictionary to store the results from every loop cycle.
        # The keys are the column names and the values are the functions we created before.
        # The functions are called with the beautiful soup object as a parameter.
        rightmove_dict = {
            'platform_id': get_ids_rm(bs)
            ,'platform': 'Rightmove'
            ,'neighbourhood': locations_dict[loc_id]
            #'overview_property_type': get_property_type_rm(bs)
            #,'bedrooms': get_bedrooms_rm(bs)
            #,'bathrooms': get_bathrooms_rm(bs)
            ,'price': get_prices_rm(bs)
            #,'title': get_title_rm(bs)
            #,'furniture': furniture    ########### avoid duplicates in the first place
            #,'furniture': get_furniture_rm(bs)
            #,'available_from': get_available_from_rm(bs)
            #,'size': get_size_rm(bs)
            #,'scraping_date': datetime.today().strftime('%Y-%m-%d')
            #,
            }
        
        # dictionary is stored in a 'temporary' dataframe within (each) loop cycle
        df_page_rm = pd.DataFrame(rightmove_dict)

        # the temporary dataframe is appended to the dataframe we set up before the loop
        df_search_rm = pd.concat([df_search_rm, df_page_rm], ignore_index=True)

    # the data frame search gets returned to the for loop to access it outside the function
    return(df_search_rm)

# see above: furnishTypes_lst = ['furnished', 'unfurnished', 'partFurnished']
# see above: rightmove_loc_id = ['5E61407', '5E61229', '5E93941', '5E61233', '5E61224', '5E93953', '5E93971', '5E61417', '5E93965']

#------------------------------------------------------------
#------------------------------------------------------------

# set up empty dataframe
df_complete_rm = pd.DataFrame()

# loop through all locations 
for loc_id in rightmove_loc_id:
    time.sleep(random.randint(2,6)/10)
    df_complete_rm = pd.concat([df_complete_rm, page_results(loc_id)], ignore_index=True)

# append column with today's date  
df_complete_rm['scraping_date'] = datetime.today().strftime('%Y-%m-%d')

#------------------------------------------------------------
#------------------------------------------------------------
# Drop IDs that are not unique (= real duplicates)
df_complete_rm.drop_duplicates(subset = ['platform_id'], inplace = True)

# convert/clean platform_id column
df_complete_rm['platform_id'] = df_complete_rm['platform_id'].str.removeprefix('property-') #.astype(int)
df_complete_rm


In [151]:
df_complete_rm

,platform_id,platform,neighbourhood,price,scraping_date
0,127749998,Rightmove,Hammersmith and Fulham,10833,2022-10-11
1,85915551,Rightmove,Hammersmith and Fulham,6500,2022-10-11
2,85912971,Rightmove,Hammersmith and Fulham,5850,2022-10-11
3,85911174,Rightmove,Hammersmith and Fulham,2297,2022-10-11
4,85909338,Rightmove,Hammersmith and Fulham,4000,2022-10-11
...,...,...,...,...,...
6395,116244392,Rightmove,Islington,3878,2022-10-11
6396,114111983,Rightmove,Islington,100,2022-10-11
6397,127055981,Rightmove,Islington,4117,2022-10-11
6398,118554665,Rightmove,Islington,2167,2022-10-11


In [112]:
# ----------------- save the dataframes into a .csv file -----------------
today = datetime.today().strftime('%Y-%m-%d-%H-%M')
df_complete_rm.to_csv('data/rightmove_overview_{}_{}.csv'.format(cycle_number, today), sep='\t')
# concatenated.to_csv('data/rightmove_detail_{}.csv'.format(today), sep='\t')
print(f"we have saved the new dataframe {cycle_number} in a .csv file")


we have saved the new dataframe 03 in a .csv file


In [113]:
# import the data frame to DBeaver

# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = f'rightmove_overview_{cycle_number}'

# import the table to sql
if engine!=None:
    try:
        df_complete_rm.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The rightmove_overview_03 table was imported successfully.


# Detail Pages 
## important: still need to implement correct order of im-/exporting from/to Database!

In [114]:
# get list of IDs from previously scraped data by querying the database
sql = f"""
SELECT platform_id 
FROM {schema}.{table_name}
"""

df_overview_ids = get_dataframe(sql)
ids = df_overview_ids['platform_id'].tolist()

print('Length of IDs list is = ' + str(len(ids)))

Length of IDs list is = 6097


In [135]:
df_details_complete = pd.DataFrame()
df_basics_complete = pd.DataFrame()
# just for testing purposes
# ids = ['127909265', '126246719']
#------------------------------------------------------------
### START OF LOOP ###
#------------------------------------------------------------
for id in ids:
    #time.sleep(random.randint(2,5)/10)
    
    # get content of detail website
    page = requests.get(f"https://www.rightmove.co.uk/properties/{id}#/?channel=RES_LET")
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')
#------------------------------------------------------------
# Part A: details
    # set up dictionary for details; fill with default values if particular detail not available on website
    bedrooms, bathrooms, size, property_type = 'NA', 'NA', 'NA', 'NA' 
    row_dict = {'detail_id':id, 'bedrooms': bedrooms, 'bathrooms': bathrooms, 'size': size, 'detailed_property_type': property_type}

    # get property details via explicit class
    details = bs.find(
        'div', class_='_4hBezflLdgDMdFtURKTWh')

    ## skip property if no details available (because property has already been removed); otherwise loop would break  
    if details == None:
        continue

    # convert details to list
    details_lst = (detail.get_text() for detail in details)
    details_lst = [detail.strip() for detail in details_lst]


    # loop through list and assign values to dictionary keys (necessary since list length is not fixed);
    # if no value is available, the default value (NA) is kept
    # (could also be done with try/except) ?!
    for detail in details_lst:
        if 'TYPE' in detail:
            row_dict['detailed_property_type'] = detail.removeprefix('PROPERTY TYPE')
        elif 'BEDROOMS' in detail:
            row_dict['bedrooms'] = detail.removeprefix('BEDROOMS×')
        elif 'BATHROOM' in detail:
            row_dict['bathrooms'] = detail.removeprefix('BATHROOMS×')
        elif 'SIZE' in detail:
            row_dict['size'] = re.search(r'\((.*?)\)', str(detail)).group(1).removesuffix(' sq. m.').replace(',', '')
    
    # store in details dataframe for this explicit id
    df_details = pd.DataFrame(row_dict, index=[0])

    # append to complete details dataframe    
    df_details_complete = pd.concat([df_details_complete, df_details], ignore_index=True)
#------------------------------------------------------------
# Part B: basics
    # via another class: get other basic details, here called "basics" (Let available from, Furnished, Title, ...)
    basics = bs.find_all(
        'div', class_='_2RnXSVJcWbWv4IpBC1Sng6')

    # convert basics to list
    # like ['Let available from:  1st May 2021', 'Deposit: Ask agent...', 'Furnished: Furnished', ...]
    basics_lst = (basic.get_text() for basic in basics)

    # split list entries by ':' and strip whitespaces. This returns List(fixed? len=5) of Lists(fixed len=2)
    # e.g. [['Let available from', ' 1st May 2021'], ['Deposit', 'Ask agent...'], ['Furnish Type', 'Furnished'], ...]
    basics_lst = [basic.strip().split(': ') for basic in basics_lst]
   
    # writes basics_lst into dict like {'Let available from': '01/08/2021', 'Furnished': 'Unfurnished', 'Title': 'Flat', ...}
    itemDict = {item[0]: item[1] for item in basics_lst}
    # add explicit id value to dict
    itemDict['platform_id'] = id

    # store basics in dataframe for this explicit id
    df_basics = pd.DataFrame(itemDict, index=[0])

    # append id-specific df to df_basics_complete containing all of already looped properties
    df_basics_complete = pd.concat([df_basics_complete, df_basics], ignore_index=True)
#------------------------------------------------------------
### END OF LOOP ###
#------------------------------------------------------------

# Part C:
# concatenate both details and basics df to new_df
new_df = pd.concat([df_details_complete, df_basics_complete], axis=1)

# pythonise column names 
new_df.rename(columns={'Let available date': 'available_from', 'Furnish type':'detailed_furniture', 'Let type': 'let_type', 'size':'size_sqm', 'Council Tax':'council_tax', 'Min. Tenancy': 'min_tenancy', 'Deposit':'deposit', 'Lease type':'lease_type'}, inplace=True)

# select those columns to keep
new_df = new_df[['detail_id', 'bedrooms', 'bathrooms', 'size_sqm', 'detailed_property_type', 'available_from', 'let_type', 'detailed_furniture', 'platform_id']]

# add date of scraping
new_df['scraping_date'] = datetime.today().strftime('%Y-%m-%d')

# rename dataframe to match table name and content
rightmove_details = new_df

# drop duplicates
# new_df.drop_duplicates(inplace=True)


# import the data frame to DBeaver
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
detail_table_name = f'rightmove_details_{cycle_number}'

# import the table to sql
if engine!=None:
    try:
        rightmove_details.to_sql(name=detail_table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {detail_table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

# first run took approx 124minutes

In [ ]:
### print items and index of ids_list

# for i in range(len(ids)):
#     if 1550 < i < 1560:
#         print(i, ids[i])

In [150]:
# ----------------- save dataframe into a .csv file -----------------
today = datetime.today().strftime('%Y-%m-%d-%H-%M')
rightmove_details.to_csv('data/rightmove_details_{}_{}.csv'.format(cycle_number, today), sep='\t')
# concatenated.to_csv('data/rightmove_detail_{}.csv'.format(today), sep='\t')
print(f"we have saved the new details dataframe {cycle_number} in a .csv file")

we have saved the new details dataframe 03 in a .csv file


In [152]:
### no need to import data from database because it is still in memory from scraping above

# sql = f"""
# SELECT * FROM {schema}.rightmove_overview_{cycle_number}
# """

# rightmove_overview = get_dataframe(sql)
# rightmove_overview

,platform_id,platform,neighbourhood,price,scraping_date
0,127749998,Rightmove,Hammersmith and Fulham,10833,2022-10-11
1,85915551,Rightmove,Hammersmith and Fulham,6500,2022-10-11
2,85912971,Rightmove,Hammersmith and Fulham,5850,2022-10-11
3,85911174,Rightmove,Hammersmith and Fulham,2297,2022-10-11
4,85909338,Rightmove,Hammersmith and Fulham,4000,2022-10-11
...,...,...,...,...,...
6092,116244392,Rightmove,Islington,3878,2022-10-11
6093,114111983,Rightmove,Islington,100,2022-10-11
6094,127055981,Rightmove,Islington,4117,2022-10-11
6095,118554665,Rightmove,Islington,2167,2022-10-11


In [159]:
# merge overview and details dataframes
rightmove_complete = pd.merge(rightmove_overview, rightmove_details, on='platform_id', how='left')
rightmove_complete

,platform_id,platform,neighbourhood,price,scraping_date_x,detail_id,bedrooms,bathrooms,size_sqm,detailed_property_type,available_from,let_type,detailed_furniture,scraping_date_y
0,127749998,Rightmove,Hammersmith and Fulham,10833,2022-10-11,127749998,5,4,271,Semi-Detached,04/12/2022,Long term,Unfurnished,2022-10-12
1,85915551,Rightmove,Hammersmith and Fulham,6500,2022-10-11,85915551,3,3,NA,Flat,Ask agent,Long term,Unfurnished,2022-10-12
2,85912971,Rightmove,Hammersmith and Fulham,5850,2022-10-11,85912971,5,3,152,Terraced,Ask agent,Long term,"Furnished or unfurnished, landlord is flexible",2022-10-12
3,85911174,Rightmove,Hammersmith and Fulham,2297,2022-10-11,85911174,2,1,NA,Flat,01/11/2022,Long term,Unfurnished,2022-10-12
4,85909338,Rightmove,Hammersmith and Fulham,4000,2022-10-11,85909338,3,NA,NA,Terraced,Now,Long term,Part furnished,2022-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,116244392,Rightmove,Islington,3878,2022-10-11,116244392,2,1,NA,Apartment,Now,Long term,Furnished,2022-10-12
6093,114111983,Rightmove,Islington,100,2022-10-11,114111983,NA,NA,NA,Parking,Now,Long term,Ask agent,2022-10-12
6094,127055981,Rightmove,Islington,4117,2022-10-11,127055981,3,2,146,Flat,Ask agent,Short term,Ask agent,2022-10-12
6095,118554665,Rightmove,Islington,2167,2022-10-11,118554665,NA,1,42,Studio,Ask agent,Short term,Ask agent,2022-10-12


In [163]:
# drop second scraping date if exists and change order of columns
rightmove_complete.rename(columns={'scraping_date_y': 'scraping_date'}, inplace=True)
rightmove_complete = rightmove_complete[['platform_id', 'platform', 'neighbourhood',  'detailed_property_type', 'size_sqm', 'bedrooms', 'bathrooms', 'price', 'detail_id', 'available_from', 'let_type', 'detailed_furniture', 'scraping_date']]


,platform_id,platform,neighbourhood,detailed_property_type,size_sqm,bedrooms,bathrooms,price,detail_id,available_from,let_type,detailed_furniture,scraping_date
0,127749998,Rightmove,Hammersmith and Fulham,Semi-Detached,271,5,4,10833,127749998,04/12/2022,Long term,Unfurnished,2022-10-12
1,85915551,Rightmove,Hammersmith and Fulham,Flat,NA,3,3,6500,85915551,Ask agent,Long term,Unfurnished,2022-10-12
2,85912971,Rightmove,Hammersmith and Fulham,Terraced,152,5,3,5850,85912971,Ask agent,Long term,"Furnished or unfurnished, landlord is flexible",2022-10-12
3,85911174,Rightmove,Hammersmith and Fulham,Flat,NA,2,1,2297,85911174,01/11/2022,Long term,Unfurnished,2022-10-12
4,85909338,Rightmove,Hammersmith and Fulham,Terraced,NA,3,NA,4000,85909338,Now,Long term,Part furnished,2022-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,116244392,Rightmove,Islington,Apartment,NA,2,1,3878,116244392,Now,Long term,Furnished,2022-10-12
6093,114111983,Rightmove,Islington,Parking,NA,NA,NA,100,114111983,Now,Long term,Ask agent,2022-10-12
6094,127055981,Rightmove,Islington,Flat,146,3,2,4117,127055981,Ask agent,Short term,Ask agent,2022-10-12
6095,118554665,Rightmove,Islington,Studio,42,NA,1,2167,118554665,Ask agent,Short term,Ask agent,2022-10-12


In [165]:
# ----------------- save dataframe into a .csv file -----------------
today = datetime.today().strftime('%Y-%m-%d-%H-%M')
rightmove_complete.to_csv('data/rightmove_complete_{}_{}.csv'.format(cycle_number, today), sep='\t')

print(f"we have saved the complete dataframe {cycle_number} in a .csv file")

we have saved the complete dataframe 03 in a .csv file


In [164]:
### upload rightmove_complete dataframe to database

# give the table a unique name
complete_table_name = f'rightmove_complete_{cycle_number}'

# import the table to sql
if engine!=None:
    try:
        rightmove_complete.to_sql(name=complete_table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {complete_table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The rightmove_complete_03 table was imported successfully.


In [177]:
#### SQL in Python ####

sql = f"""
CREATE TABLE {schema}.rightmove_complete AS
TABLE {schema}.rightmove_complete_{cycle_number};
"""
result = engine.execute(sql)

#--1
sql = f"""
UPDATE {schema}.rightmove_complete
SET  bedrooms = 0
WHERE detailed_property_type = 'Studio' ;
""" 
result = engine.execute(sql)
#----

sql = f"""
UPDATE {schema}.rightmove_complete
SET  bedrooms = NULL
WHERE bedrooms = 'NA' ;
""" 
result = engine.execute(sql)
#---
sql = f"""
UPDATE {schema}.rightmove_complete
SET bathrooms = NULL
WHERE bathrooms = 'NA';
""" 
result = engine.execute(sql)
#---

sql = f"""
UPDATE {schema}.rightmove_complete
SET size_sqm = NULL
WHERE size_sqm = 'NA';
""" 
result = engine.execute(sql)

#--8 PRICES
sql = f"""
UPDATE {schema}.rightmove_complete
SET price = NULL
WHERE price = 'POA';
"""
result = engine.execute(sql)

sql = f"""
ALTER TABLE {schema}.rightmove_complete
ALTER COLUMN price TYPE INT USING price::INT;
"""
result = engine.execute(sql)


# change column name
sql = f"""
ALTER TABLE {schema}.rightmove_complete
RENAME COLUMN detailed_property_type TO property_type;
"""
result = engine.execute(sql)


## Now: Calculations like relative prices

In [ ]:
rightmove_complete = get_dataframe(f"""SELECT * FROM {schema}.rightmove_complete;""")
rightmove_complete

In [264]:

# split size_sqm column into two columns if size is given as a range
rightmove_complete = rightmove_complete.join(rightmove_complete['size_sqm'].str.split('-', expand=True).add_prefix('size_sqm_'))

# change column types to numeric for calculation of mean size if size range is given
rightmove_complete['size_sqm_0']= pd.to_numeric(rightmove_complete['size_sqm_0'], errors='coerce')
rightmove_complete['size_sqm_1']= pd.to_numeric(rightmove_complete['size_sqm_1'], errors='coerce')

# set size value as above calculated mean size
rightmove_complete['size_sqm'] = rightmove_complete[['size_sqm_0', 'size_sqm_1']].mean(axis=1, skipna=True)

# drop size_sqm_0 and size_sqm_1 columns
rightmove_complete.drop(columns=['size_sqm_0', 'size_sqm_1'], inplace=True)

# change column type of size_sqm to numeric
rightmove_complete['size_sqm'] = rightmove_complete['size_sqm'].round()


# change column type of price and bedrooms to numeric for further calculation
rightmove_complete['price'] = pd.to_numeric(rightmove_complete['price'], errors='coerce')
rightmove_complete['bedrooms'] = pd.to_numeric(rightmove_complete['bedrooms'], errors='coerce')

In [265]:
# calculate relative price per sqm as price_sqm and skip nan values

def my_func(x):
    try:
        return int(x['price']) / int(x['size_sqm'])
    except (ZeroDivisionError, ValueError):
        return np.nan

rightmove_complete["price_sqm"] = rightmove_complete.apply(my_func, axis=1)

In [266]:
# calculate relative price per bedroom as price_bedroom and skip nan values

def my_func(x):
    try:
        return (x['price']) / (x['bedrooms'])
    except (ZeroDivisionError, ValueError):
        return (x['price'])

rightmove_complete["price_bedroom"] = rightmove_complete.apply(my_func, axis=1)

In [267]:
rightmove_complete.price_sqm = round(rightmove_complete.price_sqm, 2)
rightmove_complete.price_bedroom = round(rightmove_complete.price_bedroom, 2)

In [268]:
rightmove_complete

,platform_id,platform,neighbourhood,property_type,size_sqm,bedrooms,bathrooms,price,detail_id,available_from,let_type,detailed_furniture,scraping_date,price_sqm,price_bedroom
0,127749998,Rightmove,Hammersmith and Fulham,Semi-Detached,271.0,5.0,4,10833.0,127749998,04/12/2022,Long term,Unfurnished,2022-10-12,39.97,2166.6
1,85912971,Rightmove,Hammersmith and Fulham,Terraced,152.0,5.0,3,5850.0,85912971,Ask agent,Long term,"Furnished or unfurnished, landlord is flexible",2022-10-12,38.49,1170.0
2,85905267,Rightmove,Hammersmith and Fulham,Flat,59.0,2.0,1,2167.0,85905267,19/10/2022,Long term,Furnished,2022-10-12,36.73,1083.5
3,85511352,Rightmove,Hammersmith and Fulham,Apartment,51.0,1.0,1,4410.0,85511352,01/08/2023,Long term,Furnished,2022-10-12,86.47,4410.0
4,85897107,Rightmove,Hammersmith and Fulham,Maisonette,96.0,1.0,1,1950.0,85897107,Ask agent,Long term,Part furnished,2022-10-12,20.31,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,17284374,Rightmove,Islington,Parking,NaN,NaN,None,130.0,17284374,Ask agent,Long term,Ask agent,2022-10-12,NaN,NaN
6093,20359035,Rightmove,Islington,Parking,NaN,NaN,None,282.0,20359035,Ask agent,Long term,Ask agent,2022-10-12,NaN,NaN
6094,20829408,Rightmove,Islington,Parking,NaN,NaN,None,282.0,20829408,Ask agent,Long term,Ask agent,2022-10-12,NaN,NaN
6095,85133871,Rightmove,Islington,Parking,NaN,NaN,None,200.0,85133871,Now,Long term,Ask agent,2022-10-12,NaN,NaN


In [269]:
### upload rightmove_complete dataframe to database
## ATTENTION: We overwrite previous table with the same name and now added calculation columns

# give the table a unique name
complete_table_name = 'rightmove_complete'

# import the table to sql
if engine!=None:
    try:
        rightmove_complete.to_sql(name=complete_table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {complete_table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The rightmove_complete table was imported successfully.


## Again SQL manipulation:


In [ ]:
schema = 'capstone_jmrs'


sql = f"""
ALTER TABLE {schema}.rightmove_complete
ALTER COLUMN size_sqm TYPE INT USING size_sqm::INT,
ALTER COLUMN bedrooms TYPE INT USING bedrooms::INT,
ALTER COLUMN bathrooms TYPE INT USING bathrooms::INT,
ALTER COLUMN property_type TYPE VARCHAR,
ALTER COLUMN let_type TYPE VARCHAR,
ALTER COLUMN detailed_furniture TYPE VARCHAR,
ALTER COLUMN scraping_date TYPE DATE USING scraping_date::DATE;
"""
result = engine.execute(sql)
##-------------

sql = f"""
DELETE FROM {schema}.rightmove_complete
WHERE bedrooms IS NULL;
"""
result = engine.execute(sql)

In [ ]:
#-- ## available_from possible values: date, 'Now', 'Ask agent', NULL
#--availability
sql = f"""
ALTER TABLE {schema}.rightmove_complete
ADD available_today VARCHAR;
"""
result = engine.execute(sql)
#------------------------------------------------------------
#--####### needs to be modified (next 30 days?! #######)
sql = f"""
UPDATE {schema}.rightmove_complete
SET available_today = 'available'
WHERE available_from = 'Now';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET available_today = 'occupied'
WHERE available_from != 'Now';
"""
result = engine.execute(sql)
#--####### needs to be modified (next 30 days?! #######)
#------------------------------------------------------------
sql = f"""
UPDATE {schema}.rightmove_complete
SET available_from = NULL
WHERE available_from = 'Ask agent'; 
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET available_today = NULL
WHERE available_from = NULL;
""" 
result = engine.execute(sql)

In [279]:
sql = f"""
UPDATE {schema}.rightmove_complete
SET available_today = 'NULL'
WHERE available_from = 'NULL';
""" 
result = engine.execute(sql)

In [ ]:
sql = f"""
UPDATE {schema}.rightmove_complete
SET available_from = to_char(now(), 'DD/MM/YYYY')
WHERE available_from = 'Now';
"""
result = engine.execute(sql)
# -- 2136 Updated last time

sql = f"""
ALTER TABLE {schema}.rightmove_complete
ALTER COLUMN available_from TYPE DATE USING TO_DATE(available_from, 'DD/MM/YYYY');
"""
result = engine.execute(sql)
#----------------
#----------------

In [ ]:
sql =  f"""
DELETE FROM {schema}.rightmove_complete
WHERE detailed_furniture IS NULL
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET detailed_furniture = 'furnished'
WHERE detailed_furniture = 'Furnished';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET detailed_furniture = 'unfurnished'
WHERE detailed_furniture = 'Unfurnished';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET detailed_furniture = 'flexible'
WHERE detailed_furniture = 'Furnished or unfurnished, landlord is flexible';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET detailed_furniture = 'part furnished'
WHERE detailed_furniture = 'Part furnished';
"""
result = engine.execute(sql)
#-------------

#--Add column furniture which holds simplified furniture values
sql = f"""
ALTER TABLE {schema}.rightmove_complete
ADD COLUMN furniture VARCHAR;
"""
result = engine.execute(sql)
#---
bsql = f"""
UPDATE {schema}.rightmove_complete
SET furniture = detailed_furniture
WHERE detailed_furniture = 'furnished' OR detailed_furniture = 'unfurnished';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET furniture = 'furnished'
WHERE detailed_furniture = 'flexible';
"""
result = engine.execute(sql)

sql = f"""
UPDATE {schema}.rightmove_complete
SET furniture = 'unfurnished'
WHERE detailed_furniture = 'part furnished';
"""
result = engine.execute(sql)
#-------------
sql = f"""
DELETE FROM {schema}.rightmove_complete
WHERE furniture IS NULL;
"""
result = engine.execute(sql)
#-------------

#-- delete all rows that don't fit our targeted property_types
sql =  f"""
DELETE FROM {schema}.rightmove_complete
WHERE property_type NOT IN ('Flat', 'Apartment', 'Studio', 'Penthouse', 'Ground Flat', 'Block of Apartments')
"""
result = engine.execute(sql)

In [287]:
# create clean table
sql = f"""
CREATE TABLE {schema}.rightmove_clean AS
SELECT 
	platform_id AS platform_id, 
	platform AS platform,
	neighbourhood AS neighbourhood,
	furniture AS furniture,	
	property_type AS property_type,
	size_sqm AS size_sqm,
	bedrooms AS bedrooms,
	bathrooms AS bathrooms,
	price AS price,
	price_sqm AS price_sqm,
	price_bedroom AS price_bedroom,
	available_from AS available_from,
	available_today AS available_today,
	let_type AS let_type,
	detailed_furniture AS detailed_furniture,
	scraping_date AS scraping_date
FROM {schema}.rightmove_complete;
    """ 
result = engine.execute(sql)